In [27]:
# installing neccesory dependency
!pip install -q tensorflow-probability
!pip install -q imageio
!pip install -q git+https://github.com/tensorflow/docs

In [28]:
# importing librery
from IPython import display
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import time
import tensorflow as tf
import tensorflow_probability as tfp

In [29]:
# loading dataset
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

In [30]:
type(train_images)

numpy.ndarray

In [31]:
train_images.shape

(60000, 28, 28)

In [32]:
train_images.shape[0]

60000

In [33]:
# function for doing preprocessing
def preprocess_image(images):
  images = images.reshape(images.shape[0], 28,28,1) / 255.0 
  return np.where(images > 0.5, 1.0, 0.0)

In [34]:
train_images = preprocess_image(train_images)
test_images = preprocess_image(test_images)

In [35]:
train_images.shape

(60000, 28, 28, 1)

In [36]:
test_images.shape

(10000, 28, 28, 1)

In [37]:
# defining variables
train_size = 60000
test_size = 10000
batch_size = 32

In [39]:
# transforming numpy array to tf.dataset so it can be proceesed by tensorflow faster then shuffling and creating batch
train_dataset = (tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(test_images).shuffle(test_size).batch(batch_size) )

In [ ]:
# creating Convolutional Variational AutoEncoder
class CVAE(tf.keras.Model):
  def __int__(self, latent_dim):
    super(CVAE.self)__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(28,28,1)).
        tf.keras.layers.Conv2D(
            filters=32, kernel_size=3, strides=(2,2), activation='relu'
        ),
        tf.keras.layers.Conv2D(
            filters=64, kernel_size=3, strides=(2,2), activation='relu'
        )
        tf.keras.layers.Flatten()
        # No activation
        tf.keras.layers.Dense(latent_dim + latent_dim)
    ])
  self.decoder = tf.keras.layers.Sequential([
      tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
      tf.keras.layers.Dense(units=7*7*32, activation='relu'),
      tf.keras.layers.Reshape(target_shape=(7,7,32)),
      tf.keras.layers.Conv2DTranspose(
          filters=64, kernel_size=3, strides=2, padding='same',
          activation='relu'
      )
      tf.keras.layers.Conv2DTranspose(
          filters=32, kernel_size=3, strides=2, padding='same'),
          activation='relu'
      ),
      tf.keras.layers.Conv2DTranspose(
          filters=1, kernel_size=3, strides=1, padding='same'
      )
  ])

  @tf.function
  def sample(self, eps=None):
    if eps in None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)
  
  